📑**Project Brief — Multi-Agent NSE Stock Research (India Edition)**

This project is a lightweight, agentic research assistant for Indian equities (NSE). It automates a typical equity-research workflow:

Pulls prices (tries yfinance, falls back to a realistic synthetic series so it always runs).

Processes India-focused news (mock feed; no API keys needed).

Adds macro context (mock CPI/WPI/Repo/INR).

Routes information to simple specialist agents (Earnings/News/Market).

Drafts a note, self-evaluates (coverage, consistency, actionability), and refines a final report.

Saves a tiny memory per ticker across runs.

The result: a compact research report with charts, KPIs, and clear monitor/risks—no setup headaches.


NSE Ticker ──► InvestmentResearchAgent
                ├─► Finance Tool (yfinance or synthetic)
                ├─► News Tool (India mock)
                ├─► Macro Tool (CPI/WPI/Repo/INR mock)
                └─► Router ──► EarningsAgent / NewsAgent / MarketAgent
                                   │
                  Draft ──► Evaluator (coverage/consistency/actionability)
                            └─► Optimizer (auto-refinements)
                               └─► Final Report + Memory

**Agents**

InvestmentResearchAgent: plans steps, calls tools, composes the draft, triggers evaluation and refinement, stores run memory.

EarningsAgent: reads EPS/guidance/capex/ARPU cues from snippets.

NewsAgent: prompt chaining → ingest → preprocess → classify sentiment → extract signals (RBI/SEBI/PLI) → summarize tilt.

MarketAgent: adds macro snapshot + crude “regime” tag.

**Key Patterns**

Prompt Chaining (News): Ingest → Clean → Classify → Extract → Summarize.

Routing: Snippets to earnings vs news specialists.

Evaluator–Optimizer: Score the draft → apply feedback → append missing context + explicit monitor/risks.

In [11]:
# ======================================================================
# 🇮🇳 Multi-Agent NSE Research — Single Cell (Clean, Fixed, Beautiful)
# - Safe scalar extraction (.iloc[-1]) everywhere
# - No Series truth-testing in any `if`
# - yfinance(auto_adjust=False) to avoid FutureWarning
# - Timezone-aware UTC timestamps for memory
# - Works offline (synthetic prices + mock India news/macro)
# ======================================================================

# ── 0) USER INPUT (clearly demarcated) ─────────────────────────────────
DEFAULT_TICKER = "RELIANCE.NS"  # e.g., HDFCBANK.NS, INFY.NS, TCS.NS  (keep .NS)
DEFAULT_PERIOD = "6mo"          # "3mo", "6mo", "1y"
SUGGESTED_TICKERS = [
    "RELIANCE.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "SBIN.NS", "LT.NS", "BHARTIARTL.NS", "ITC.NS", "ASIANPAINT.NS"
]

# ── 1) Imports & Setup ────────────────────────────────────────────────
from __future__ import annotations
import os, json, re, random, sys, subprocess
from typing import Dict, Any, List, Optional, Tuple
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# Optional widgets for nicer inputs
try:
    import ipywidgets as widgets
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False

# Optional yfinance; we will fall back if unavailable
try:
    import yfinance as yf
    YF_AVAILABLE = True
except Exception:
    YF_AVAILABLE = False

pd.set_option("display.width", 120)
pd.set_option("display.max_colwidth", 120)

# ── 2) Global Styles & Banners ────────────────────────────────────────
def inject_global_style() -> None:
    css = """
    <style>
      @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
      * { font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, Cantarell, "Helvetica Neue", Arial, sans-serif !important; }
      .hero { padding:20px; border-radius:16px; color:#fff; background:linear-gradient(90deg,#0ea5e9,#22c55e); box-shadow:0 8px 24px rgba(0,0,0,0.12); margin:6px 0 14px; }
      .hero h1 { margin:0; font-size:30px; letter-spacing:0.2px; }
      .hero p  { margin:8px 0 0; font-size:16px; }
      .note { border:1px solid #e5e7eb; border-left:8px solid var(--bar,#22c55e); background:var(--bg,#f8fff9); border-radius:12px; padding:12px 14px; margin:12px 0; }
      .card { border:1px solid #e5e7eb; border-radius:14px; padding:16px; margin:12px 0; background:#fff; box-shadow:0 2px 8px rgba(0,0,0,0.05); }
      .h2 { font-size:20px; font-weight:700; margin:6px 0 10px; }
      .kpis { display:flex; gap:10px; flex-wrap:wrap; margin-top:8px; }
      .chip { border-radius:999px; padding:8px 14px; font-weight:700; letter-spacing:.2px; background:#f2fbff; border:1px solid #d7eefc; }
      .chip.ok { background:#f8fff9; border-color:#dcfce7; }
      .chip.bad { background:#fff5f5; border-color:#ffdede; }
      .mono { font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", monospace !important; }
      .input-panel { border:2px dashed #0ea5e9; background:#f2fbff; border-radius:12px; padding:14px; }
    </style>
    """
    display(HTML(css))

def banner(msg: str, kind: str = "info") -> None:
    colors = {
        "info": ("#0ea5e9", "#f2fbff"),
        "ok":   ("#22c55e", "#f8fff9"),
        "warn": ("#f59e0b", "#fffaf2"),
        "err":  ("#ef4444", "#fff2f2"),
    }
    border, bg = colors.get(kind, colors["info"])
    display(HTML(f"<div class='note' style='--bar:{border};--bg:{bg}'>{msg}</div>"))

inject_global_style()
display(HTML("""
<div class='hero'>
  <h1>🇮🇳 Multi-Agent NSE Stock Research</h1>
  <p>Pick a stock, click <b>Run</b>, get a visual research note. No API keys needed.</p>
</div>
<div class='note'>Under the hood: prices (or synthetic), India-focused news & macro (mock), specialist routing, self-evaluation, auto-refinement, and tiny per-ticker memory.</div>
"""))

# ── 3) Utilities (safe scalars / returns / time) ──────────────────────
def to_float(x: Any) -> Optional[float]:
    """Convert 0-d np/pandas scalars (or plain numbers) to float; return None if not possible."""
    try:
        if x is None: return None
        if isinstance(x, (int, float, np.floating)): return float(x)
        # pandas Series of length 1
        if hasattr(x, "ndim") and getattr(x, "ndim", None) == 1 and len(x) == 1:
            return float(x.iloc[0])
        # pandas Series/array scalar
        if hasattr(x, "item"):
            return float(x.item())
        return float(x)
    except Exception:
        return None

def utc_timestamp() -> str:
    return dt.datetime.now(dt.timezone.utc).isoformat(timespec="seconds").replace("+00:00", "Z")

def compute_returns(close_series: pd.Series) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    """Compute approx 1M/3M/6M pct returns safely (in %)."""
    try:
        s = pd.to_numeric(close_series, errors="coerce").dropna().reset_index(drop=True)
        n = len(s)
        if n == 0: return (None, None, None)
        last = float(s.iloc[-1])
        def _pct(days: int) -> Optional[float]:
            idx = max(0, n - days)
            base = float(s.iloc[idx])
            if base == 0: return None
            return round(100.0 * (last / base - 1.0), 2)
        r1 = _pct(21)
        r3 = _pct(63)
        r6 = _pct(126 if n >= 126 else n-1)
        return (r1, r3, r6)
    except Exception:
        return (None, None, None)

# ── 4) Tiny JSON Memory ───────────────────────────────────────────────
MEMORY_PATH = "/content/agent_memory_nse.json"
def load_memory(path: str = MEMORY_PATH) -> Dict[str, Any]:
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return {"notes": {}, "runs": 0}
    return {"notes": {}, "runs": 0}

def save_memory(memory: Dict[str, Any], path: str = MEMORY_PATH) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(memory, f, indent=2)

def memorize(ticker: str, note: str) -> None:
    m = load_memory()
    m.setdefault("notes", {}).setdefault(ticker.upper(), [])
    if note not in m["notes"][ticker.upper()]:
        m["notes"][ticker.upper()].append(note)
    m["runs"] = int(m.get("runs", 0)) + 1
    save_memory(m)

def recall(ticker: str) -> List[str]:
    m = load_memory()
    return m.get("notes", {}).get(ticker.upper(), [])

# ── 5) Mock India News & Macro (offline-friendly) ─────────────────────
MOCK_NEWS_INDIA = [
    {"title": "RBI keeps repo rate unchanged; stance stays withdrawal of accommodation", "published": "2025-09-10",
     "content": "Sticky food inflation noted; liquidity fine-tuning to continue; commentary supportive of growth.", "type": "macro"},
    {"title": "Reliance beats estimates on retail and Jio growth", "published": "2025-08-01",
     "content": "Consolidated EPS beat by ₹2; ARPU up; retail footfalls robust; O2C margins stable.", "type": "earnings"},
    {"title": "SEBI proposes tweaks to related-party norms for large conglomerates", "published": "2025-07-25",
     "content": "Consultation paper suggests higher disclosure thresholds; potential compliance costs.", "type": "regulatory"},
    {"title": "Green energy capex ramps up across top corporates", "published": "2025-09-14",
     "content": "Incremental capex toward solar PV and battery storage; margin expansion expected over FY26-27.", "type": "capex"},
]

MOCK_MACRO_INDIA = {
    "CPI_YOY_IND": [
        {"date": "2025-05-01", "value": 5.1},
        {"date": "2025-06-01", "value": 5.0},
        {"date": "2025-07-01", "value": 4.9},
        {"date": "2025-08-01", "value": 4.8},
    ],
    "WPI_YOY": [
        {"date": "2025-05-01", "value": 1.8},
        {"date": "2025-06-01", "value": 2.1},
        {"date": "2025-07-01", "value": 2.3},
        {"date": "2025-08-01", "value": 2.2},
    ],
    "RBI_REPO": [
        {"date": "2025-07-01", "value": 6.50},
        {"date": "2025-08-01", "value": 6.50},
        {"date": "2025-09-01", "value": 6.50},
    ],
    "USDINR": [
        {"date": "2025-06-01", "value": 84.2},
        {"date": "2025-07-01", "value": 83.9},
        {"date": "2025-08-01", "value": 83.6},
    ]
}

# ── 6) Tools (finance/news/macro) with robust fallbacks ───────────────
def tool_finance_prices_nse(ticker: str, period: str = "6mo") -> pd.DataFrame:
    """OHLCV via yfinance if possible; otherwise produce a synthetic daily series."""
    t = (ticker or "").upper()
    if YF_AVAILABLE and t:
        try:
            # Keep raw OHLCV; avoid auto-adjust warning
            df = yf.download(t, period=period, interval="1d", auto_adjust=False, progress=False)
            if isinstance(df, pd.DataFrame) and (not df.empty):
                out = df.reset_index().rename(columns={"Date": "date"}).copy()
                # Ensure schema
                if "date" not in out.columns:
                    # If Date did not appear (rare), coerce index
                    out = out.reset_index().rename(columns={out.columns[0]: "date"})
                out["date"] = pd.to_datetime(out["date"])
                for col in ["Open","High","Low","Close","Adj Close","Volume"]:
                    if col in out.columns:
                        out[col] = pd.to_numeric(out[col], errors="coerce")
                return out
        except Exception:
            pass
    # Synthetic fallback (always-runs)
    rng = pd.date_range(end=pd.Timestamp.today().normalize(), periods=126, freq="B")
    base = 2500.0
    rows, level = [], base
    for i, d in enumerate(rng):
        drift = 0.0003
        shock = random.gauss(0, 0.008)
        if i % 21 == 0:
            shock += random.choice([0.015, -0.015, 0])
        level *= (1 + drift + shock)
        high = level * (1 + abs(random.gauss(0, 0.004)))
        low  = level * (1 - abs(random.gauss(0, 0.004)))
        open_ = level * (1 + random.gauss(0, 0.002))
        close = level
        vol   = abs(int(random.gauss(8e6, 2e6)))
        rows.append({"date": d, "Open": open_, "High": high, "Low": low,
                     "Close": close, "Adj Close": close, "Volume": vol})
    return pd.DataFrame(rows)

def tool_news_search_india(ticker: str, limit: int = 8) -> List[Dict[str, Any]]:
    return list(MOCK_NEWS_INDIA[:limit])

def tool_macro_india(series_id: str) -> pd.DataFrame:
    """Return a DataFrame with datetime 'date' and numeric 'value' columns."""
    data = MOCK_MACRO_INDIA.get(series_id, [])
    df = pd.DataFrame(data)
    if not df.empty and "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
    if "value" in df.columns:
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df

# ── 7) News Prompt-Chaining (ingest → preprocess → classify → extract → summarize) ──
BULLISH_KW = {"beat", "expansion", "tailwinds", "improved", "higher", "record", "gains", "growth", "arpu", "pli"}
BEARISH_KW = {"miss", "softness", "weak", "fine", "inquiry", "investigation", "headwind", "decline", "inflation"}

def ingest_news_raw_india(ticker: str) -> List[Dict[str, Any]]:
    return tool_news_search_india(ticker)

def preprocess_articles_india(articles: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    out: List[Dict[str, Any]] = []
    for a in articles:
        title = str(a.get("title","")).strip()
        content = str(a.get("content","")).strip()
        text = re.sub(r"\s+", " ", f"{title}\n{content}").lower()
        out.append({**a, "text": text})
    return out

def classify_sentiment(text: str) -> str:
    bull = sum(1 for k in BULLISH_KW if k in text)
    bear = sum(1 for k in BEARISH_KW if k in text)
    if bull > bear: return "bullish"
    if bear > bull: return "bearish"
    return "neutral"

def extract_signals_india(text: str) -> Dict[str, bool]:
    return {
        "mentions_rbi":  bool(re.search(r"\brbi\b|repo", text, re.I)),
        "mentions_sebi": bool(re.search(r"\bsebi\b|related-party|disclosure", text, re.I)),
        "mentions_pli":  bool(re.search(r"\bpli\b|production linked incentive", text, re.I)),
    }

def summarize_articles_india(items: List[Dict[str, Any]]) -> str:
    if not items: return "No recent articles in mock India feed."
    lines = [f"• {a.get('published','?')}: {a.get('title','?')} → sentiment={a.get('sentiment','?')}, signals={a.get('signals',{})}" for a in items]
    bulls = sum(1 for a in items if a.get('sentiment') == 'bullish')
    bears = sum(1 for a in items if a.get('sentiment') == 'bearish')
    tilt = "bullish" if bulls > bears else ("bearish" if bears > bulls else "mixed")
    lines.append(f"Overall news tilt: {tilt} (bullish={bulls}, bearish={bears}).")
    return "\n".join(lines)

def news_pipeline_india(ticker: str) -> Dict[str, Any]:
    raw = ingest_news_raw_india(ticker)
    proc = preprocess_articles_india(raw)
    for a in proc:
        a["sentiment"] = classify_sentiment(a["text"])
        a["signals"]   = extract_signals_india(a["text"])
    return {"articles": proc, "summary": summarize_articles_india(proc)}

# ── 8) Specialists & Router ───────────────────────────────────────────
def earnings_agent(snippet: str) -> Dict[str, bool]:
    beat      = bool(re.search(r"\bbeat|above expectations|estimates", snippet, re.I))
    miss      = bool(re.search(r"\bmiss|below expectations|lagged", snippet, re.I))
    guidance  = bool(re.search(r"guidance|outlook|capex|arpu", snippet, re.I))
    return {"eps_beat": (beat and not miss), "eps_miss": (miss and not beat), "guidance_mentioned": guidance}

def market_agent_india() -> Dict[str, Any]:
    """Return scalar macro snapshot; never returns pandas objects."""
    cpi_df    = tool_macro_india("CPI_YOY_IND")
    wpi_df    = tool_macro_india("WPI_YOY")
    repo_df   = tool_macro_india("RBI_REPO")
    usdinr_df = tool_macro_india("USDINR")
    cpi    = to_float(cpi_df["value"].iloc[-1])    if (isinstance(cpi_df, pd.DataFrame) and not cpi_df.empty) else None
    wpi    = to_float(wpi_df["value"].iloc[-1])    if (isinstance(wpi_df, pd.DataFrame) and not wpi_df.empty) else None
    repo   = to_float(repo_df["value"].iloc[-1])   if (isinstance(repo_df, pd.DataFrame) and not repo_df.empty) else None
    usdinr = to_float(usdinr_df["value"].iloc[-1]) if (isinstance(usdinr_df, pd.DataFrame) and not usdinr_df.empty) else None
    # Explicit checks only on plain floats
    regime = "disinflation drift"
    if (cpi is not None) and np.isfinite(cpi) and (cpi > 5.0):
        regime = "inflation watch"
    return {"cpi_yoy": cpi, "wpi_yoy": wpi, "repo": repo, "usdinr": usdinr, "regime": regime}

def route_snippet_india(snippet: str) -> str:
    s = snippet.lower()
    return "earnings" if any(k in s for k in ["eps", "earnings", "guidance", "revenue", "arpu", "capex"]) else "news"

# ── 9) Evaluator → Optimizer (self-reflection) ────────────────────────
def evaluate_report(draft: str, have_prices: bool, have_news: bool, have_macro: bool) -> Dict[str, Any]:
    cov = (1 if have_prices else 0) + (1 if have_news else 0) + (1 if have_macro else 0)
    coverage    = cov / 3.0
    consistency = 1.0 if (len(draft) > 200 and draft.count("\n") >= 3) else 0.6
    actionable  = 1.0 if bool(re.search(r"(watch|consider|risk|catalyst|monitor|budget|fpi)", draft, re.I)) else 0.5
    score       = round((coverage + consistency + actionable) / 3.0, 2)
    feedback: List[str] = []
    if not have_prices: feedback.append("Add a price/returns section with a simple chart.")
    if not have_news:   feedback.append("Include a concise news summary with sentiment tilt.")
    if not have_macro:  feedback.append("Mention macro context (CPI/WPI/Repo/INR) and a regime tag.")
    if actionable < 1.0: feedback.append("Add explicit monitoring points: RBI policy, SEBI rules, FPI flows, monsoon impact.")
    return {"score": score, "coverage": coverage, "consistency": consistency, "actionable": actionable, "feedback": feedback}

def optimize_report(draft: str, ev: Dict[str, Any], news_summary: str, macro: Dict[str, Any]) -> str:
    refined = draft
    if ev.get("feedback"):
        refined += "\n\n---\n**Refinements based on evaluator feedback:**\n" + "\n".join(f"- {fb}" for fb in ev["feedback"])
    if ("news summary" not in draft.lower()) and news_summary:
        refined += f"\n\n**News Summary (auto-added):**\n{news_summary}\n"
    if ("macro context" not in draft.lower()) and macro:
        refined += (f"\n**Macro Context (auto-added):** CPI ~ {macro.get('cpi_yoy','?')}%, "
                    f"WPI ~ {macro.get('wpi_yoy','?')}%, RBI Repo ~ {macro.get('repo','?')}%, "
                    f"USDINR ~ {macro.get('usdinr','?')}, Regime: {macro.get('regime','?')}.\n")
    refined += "\n**Monitor:** RBI policy, SEBI consultations, FPI net flows, monsoon & food inflation."
    refined += "\n**Risks:** regulatory changes, demand slowdown, commodity volatility, INR depreciation."
    return refined

# ── 10) Visualization helpers ─────────────────────────────────────────
def render_kpis(ticker: str, r1m: Optional[float], r3m: Optional[float], r6m: Optional[float]) -> None:
    def cls(v): return "chip ok" if (v is not None and v >= 0) else ("chip bad" if v is not None else "chip")
    html = f"""
    <div class='card'>
      <div class='h2'>Snapshot • {ticker}</div>
      <div class='kpis'>
        <div class='{cls(r1m)}'>1M: {r1m if r1m is not None else '—'}%</div>
        <div class='{cls(r3m)}'>3M: {r3m if r3m is not None else '—'}%</div>
        <div class='{cls(r6m)}'>6M: {r6m if r6m is not None else '—'}%</div>
      </div>
    </div>
    """
    display(HTML(html))

def plot_price_chart(df: pd.DataFrame, ticker: str) -> None:
    plt.figure(figsize=(10,4))
    plt.plot(df["date"], df["Close"])
    plt.title(f"{ticker} — Close Price (last ~6 months)")
    plt.xlabel("Date"); plt.ylabel("Close (₹)"); plt.tight_layout(); plt.show()

def plot_sentiment_bars(articles: List[Dict[str, Any]]) -> None:
    counts = {"bullish":0, "neutral":0, "bearish":0}
    for a in articles:
        key = a.get("sentiment", "neutral")
        counts[key] = counts.get(key, 0) + 1
    labels = list(counts.keys())
    vals = [counts[k] for k in labels]
    plt.figure(figsize=(6,3.2)); plt.bar(labels, vals); plt.title("News Sentiment (mock)")
    plt.tight_layout(); plt.show()

def plot_macro_bars(macro: Dict[str, Any]) -> None:
    labels = ["CPI YoY", "WPI YoY", "Repo", "USDINR"]
    vals = [macro.get("cpi_yoy"), macro.get("wpi_yoy"), macro.get("repo"), macro.get("usdinr")]
    plt.figure(figsize=(7.5,3.2)); plt.bar(labels, vals); plt.title("Macro Snapshot (latest)")
    plt.tight_layout(); plt.show()

def render_eval_table(ev: Dict[str, Any]) -> None:
    html = f"""
    <div class='card'>
      <div class='h2'>Evaluator Scores <span class='mono' style='font-weight:700'>score: {ev.get('score','?')}</span></div>
      <table style="width:100%;border-collapse:collapse;font-size:14px">
        <tr><td style="padding:6px;border-bottom:1px solid #eee">Coverage</td><td class="mono" style="padding:6px;border-bottom:1px solid #eee">{round(ev.get('coverage',0)*100,1)}%</td></tr>
        <tr><td style="padding:6px;border-bottom:1px solid #eee">Consistency</td><td class="mono" style="padding:6px;border-bottom:1px solid #eee">{round(ev.get('consistency',0)*100,1)}%</td></tr>
        <tr><td style="padding:6px;border-bottom:1px solid #eee">Actionability</td><td class="mono" style="padding:6px;border-bottom:1px solid #eee">{round(ev.get('actionable',0)*100,1)}%</td></tr>
      </table>
      <div style="margin-top:8px"><b>Feedback:</b><br><span class="mono">{'<br>'.join(ev.get('feedback', []) or ['—'])}</span></div>
    </div>
    """
    display(HTML(html))

def render_final_report(text: str) -> None:
    html = f"""
    <div class='card'>
      <div class='h2'>Final Research Report</div>
      <details open>
        <summary>Show/Hide report</summary>
        <pre class="mono" style="white-space:pre-wrap;font-size:13px;margin-top:8px">{text}</pre>
      </details>
    </div>
    """
    display(HTML(html))

# ── 11) Main Agent ────────────────────────────────────────────────────
class InvestmentResearchAgent:
    """Plans → gathers → routes → drafts → evaluates → optimizes → remembers."""
    def __init__(self, name: str = "Analyst-India"):
        self.name = name

    def plan(self, ticker: str) -> List[str]:
        return [
            f"Fetch prices for {ticker}",
            f"Fetch India-focused news for {ticker}",
            "Assess macro background (CPI/WPI/Repo/INR)",
            "Route items to specialists and aggregate signals",
            "Draft report → Evaluate → Optimize",
        ]

    def run(self, ticker: str, period: str = "6mo") -> Dict[str, Any]:
        tk = (ticker or "").upper()
        steps  = self.plan(tk)

        prices = tool_finance_prices_nse(tk, period)
        have_prc = bool(isinstance(prices, pd.DataFrame) and (not prices.empty))

        news_out = news_pipeline_india(tk)
        have_news = bool(news_out.get("articles"))  # list truthiness → explicit bool

        macro = market_agent_india()
        have_macro = bool(isinstance(macro, dict) and (len(macro) > 0))

        # Routed samples
        routed: List[Dict[str, Any]] = []
        for a in news_out.get("articles", [])[:3]:
            route = route_snippet_india(a.get("text", ""))
            routed.append({"title": a.get("title",""), "route": route,
                           "result": (earnings_agent(a["text"]) if route=="earnings"
                                      else {"sentiment": a.get("sentiment"), **a.get("signals", {})})})

        # Returns
        r1, r3, r6 = (None, None, None)
        try:
            r1, r3, r6 = compute_returns(prices["Close"])
        except Exception:
            pass

        memory_notes = recall(tk)
        draft = (
            f"### {tk} Research Note (India)\n"
            f"**Agent:** {self.name}\n\n"
            f"**Plan:** {', '.join(steps)}\n\n"
            f"**Prices available:** {have_prc}; **News items:** {len(news_out.get('articles', []))}; **Macro available:** {have_macro}.\n\n"
            f"**Simple Returns:** 1M={r1}%, 3M={r3}%, 6M={r6}% (approx).\n\n"
            f"**Routed Signals (samples):** {routed}\n\n"
            f"**News Summary:**\n{news_out.get('summary','—')}\n\n"
            f"**Macro Context:** CPI ~ {macro.get('cpi_yoy','?')}%, WPI ~ {macro.get('wpi_yoy','?')}%, "
            f"RBI Repo ~ {macro.get('repo','?')}%, USDINR ~ {macro.get('usdinr','?')}, Regime: {macro.get('regime','?')}.\n\n"
            f"**Memory Notes:** {memory_notes if memory_notes else '—'}\n\n"
            f"**Preliminary View:** Consider impacts of RBI policy stability, SEBI disclosures, and capex cycle on multiples.\n"
        )

        ev = evaluate_report(draft, have_prc, have_news, have_macro)
        final = optimize_report(draft, ev, news_out.get("summary",""), macro)

        ts = utc_timestamp()
        memorize(tk, f"Run at {ts} → score {ev['score']}")

        return {"ticker": tk, "prices": prices, "news": news_out, "macro": macro,
                "returns": {"1M": r1, "3M": r3, "6M": r6},
                "evaluation": ev, "final_report": final}

# ── 12) Runner + UI ───────────────────────────────────────────────────
def run_analysis(ticker: str, period: str = "6mo") -> None:
    display(HTML(f"<div class='input-panel'><b>Running</b> for <span class='mono'>{ticker}</span> • Period: <span class='mono'>{period}</span></div>"))
    agent = InvestmentResearchAgent(name="Analyst-India-Colab")
    out   = agent.run(ticker, period=period)

    # KPIs
    r = out["returns"]; render_kpis(out["ticker"], r.get("1M"), r.get("3M"), r.get("6M"))

    # Price chart
    display(HTML("<div class='h2'>Price Chart</div>"))
    plot_price_chart(out["prices"], out["ticker"])

    # Sentiment + Macro
    display(HTML("<div class='h2'>News Sentiment</div>")); plot_sentiment_bars(out["news"]["articles"])
    display(HTML("<div class='h2'>Macro Snapshot</div>")); plot_macro_bars(out["macro"])

    # Evaluator & Final Report
    render_eval_table(out["evaluation"]); render_final_report(out["final_report"])

# ── 13) Controls (Dropdown if available; else defaults) ───────────────
if HAS_WIDGETS:
    display(HTML("<div class='card'><div class='h2'>Step 1 — Pick your NSE stock (keep .NS)</div></div>"))
    dd_ticker = widgets.Combobox(
        placeholder="Type or pick an NSE ticker",
        options=SUGGESTED_TICKERS,
        value=DEFAULT_TICKER,
        description="Ticker:",
        ensure_option=False,
        layout=widgets.Layout(width="360px"),
        style={"description_width": "70px"}
    )
    dd_period = widgets.Dropdown(
        options=["3mo", "6mo", "1y"],
        value=DEFAULT_PERIOD,
        description="Period:",
        layout=widgets.Layout(width="180px"),
        style={"description_width": "70px"}
    )
    run_btn = widgets.Button(description="Run Analysis", button_style="primary")
    ui_box = widgets.HBox([dd_ticker, dd_period, run_btn])
    out_box = widgets.Output()
    display(ui_box, out_box)

    def _on_click(_):
        with out_box:
            out_box.clear_output(wait=True)
            run_analysis((dd_ticker.value or "").strip() or DEFAULT_TICKER, dd_period.value)

    run_btn.on_click(_on_click)

    # Auto-run once with defaults
    with out_box:
        run_analysis(DEFAULT_TICKER, DEFAULT_PERIOD)
else:
    banner("Widgets unavailable — running with defaults. Edit DEFAULT_TICKER/DEFAULT_PERIOD at the top if needed.", "warn")
    run_analysis(DEFAULT_TICKER, DEFAULT_PERIOD)

# ── 14) Optional Export (HTML → print to PDF) ─────────────────────────
def export_notebook_to_html(ipynb_path: str, output_path: str = "NSE_Research_Report.html") -> None:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "nbconvert", "jupyter"])
    except Exception:
        pass
    cmd = [sys.executable, "-m", "jupyter", "nbconvert", "--to", "html", ipynb_path, "--output", output_path]
    try:
        subprocess.check_call(cmd)
        banner(f"Exported HTML → <span class='mono'>{output_path}</span>", "ok")
    except Exception as e:
        banner(f"Export failed: <span class='mono'>{e}</span>", "err")


Output()